# Webscrape press releases of HKGOV from Jan to June, 2020.

Tools: Seleium, BeautifullSoup, Regex, Pandas, Numpy

In [51]:
#import library we need
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup as bs

import requests
import pandas as pd
import re
import numpy as np

In [ ]:
# set Chrome Options
chrome_options = Options() 
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

# define Chrome webdriver
driver = webdriver.Chrome(r'C:\Users\xxx',options=chrome_options)


## Set up a root link of the website & other details for later

Sample link for reference ==>  https://www.info.gov.hk/gia/general/202001/09/P2020010900907.htm

URL Analysis

root link --> https://www.info.gov.hk/gia/general/

month     --> 202001/

date      --> 09/

press ID  --> P2020010900907

Idea is very simple, we get press IDs of everyday first. Then we scape contents of each page.

In [4]:
URL = 'https://www.info.gov.hk/'
datedict = {'1':31, '2':29, '3':31, '4':30, '5':31, '6':30}


In [60]:
# set up a empty list for data storage
l=[]


for i in datedict:
    for j in range(1,datedict[i]+1):
        if j in range(1,10):
            j = '0'+str(j)
        url = URL +'gia/general/20200'+i+'/'+str(j)+'.htm'
        driver.get(url)
        waitxpath = '//*[@id="contentBody"]/div[1]/div[1]/ul/li[1]/a'
        try:
            element_present = EC.presence_of_element_located((By.XPATH, waitxpath))
            WebDriverWait(driver, 10).until(element_present)
        except:
            print(driver.current_url, 'die')
            break
        date_url_list=bs(requests.get(driver.current_url).text, 'html.parser').find_all(class_='NEW')
        for k in date_url_list:
            d={}
            new_url = URL[:-1]+k['href']
            soup = bs(requests.get(new_url).text, 'html.parser')
            
            trial = 1
            while not soup.find(id="PRHeadlineSpan"):
                soup = bs(requests.get(new_url).text, 'html.parser')
                print(new_url, trial ,'times try')
                trial += 1
                
            d['Date']='2020-'+i+'-'+str(j)
            try:
                d['time']=re.search(r'HKT \d\d:\d\d', soup.find(id="pressrelease").text).group()
            except:
                d['time']=np.nan
            d['Title']=soup.find(id="PRHeadlineSpan").text
            d['Content']=soup.find(id="pressrelease").text.replace('\n','').replace('\t','').replace('\xa0','').replace('â\x80\x8b','').replace('\r','')
            d['url']=new_url
            l.append(d)
            
            

https://www.info.gov.hk/gia/general/202005/30/P2020053000753.htm 1 times try


## Save data to csv format.

In [61]:
df = pd.DataFrame(l)
df.to_csv('gov_2020_jan-jun_news.csv')

## Number of rows in DataFrame

In [62]:
len(l)

4788